# Import Libraries

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy

**"About this file" (source : Kaggle)**

This dataset contains about **10 years of daily weather observations** from numerous **Australian weather stations.**

**RainTomorrow** is the **target variable to predict.** It means -- **did it rain the next day, Yes or No?**

This column is Yes if the rain for that day was 1mm or more.

# I- LSTM (Best accuracy : 88.99%)

## A. Preprocessing

### The goal is to have 1 continuous time slot $[D_{start},D_{end}]$ without missing days for all the cities. It will allow a sequential process of all the data by the LSTM model.

In [2]:
data = pd.read_csv("weatherAUS.csv")
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['cos_time'] = np.cos(2*np.pi*(data['Day']+30.5*(data['Month']-1))/365)
data['sin_time'] = np.sin(2*np.pi*(data['Day']+30.5*(data['Month']-1))/365)
data.drop(['Month','Day','Year'], axis=1, inplace = True)

data = data[data["RainTomorrow"].notna()]
data['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
data['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

In [3]:
data.dropna(inplace=True) #On perd complètement les données de plusieurs villes.

In [4]:
data.set_index('Date', inplace=True)

In [5]:
data = data[data['Location'] != 'Cobar'] #Car on n'a des données que entre 2009 et 2010
data = data[data['Location'] != 'CoffsHarbour'] #Car on n'a des données que entre 2009 et 2014
data = data[data['Location'] != 'Sale'] #Car on n'a des données que entre 2009 et 2014
data = data[data['Location'] != 'Canberra'] #Car on n'a des données que entre 2009 et 2012

In [6]:
nb_lieux = data['Location'].unique().shape[0]
print(f'On conserve les données de {nb_lieux} villes')

On conserve les données de 22 villes


In [7]:
#On garde la fourchette de temps dans laquelle on a le moins de données manquantes
start_date = max([data[data['Location'] == location].index.min() for location in data['Location'].unique()])
end_date = min([data[data['Location'] == location].index.max() for location in data['Location'].unique()])

In [8]:
#On garde seulement les données entre start_date et end_date
data = data[data.index >= start_date]
data = data[data.index <= end_date]

In [9]:
#On ajoute les jours manquants
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
all_locations = data['Location'].unique()
full_data = []
for location in all_locations:
    data_location = data[data['Location'] == location]
    data_location = data_location.reindex(all_dates)
    #On fait de l'interpolation pour les variables continues
    data_location = data_location.interpolate(method='time')
    #On met la dernière valeur non nulle pour les variables catégorielles
    data_location.fillna(method='ffill', inplace=True)
    data_location.drop('Location', axis=1, inplace=True)
    data_location['RainToday'] = data_location['RainToday'].astype('int64')
    data_location['RainTomorrow'] = data_location['RainTomorrow'].astype('int64')
    full_data.append(data_location)
#full_data = pd.concat(full_data)

/var/folders/yw/kt9c2n_149ncnw417n00tw640000gn/T/ipykernel_84878/3196589438.py:9: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data_location = data_location.interpolate(method='time')
/var/folders/yw/kt9c2n_149ncnw417n00tw640000gn/T/ipykernel_84878/3196589438.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_location.fillna(method='ffill', inplace=True)
/var/folders/yw/kt9c2n_149ncnw417n00tw640000gn/T/ipykernel_84878/3196589438.py:9: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data_location = data_location.interpolate(method='time')
/var/folders/yw/kt9c2n_149ncnw417n00tw640000gn/T/ipykernel_84878/3196589438.py:11: FutureWarning: DataFram

In [10]:
np.array(full_data).shape

(22, 2016, 23)

In [11]:
print(f'full_data gives, for 22 cities, a row of information for each day in [{start_date},{end_date}]')
print(f'We have {full_data[0].shape[0]} rows per city.')

full_data gives, for 22 cities, a row of information for each day in [2010-10-20 00:00:00,2016-04-26 00:00:00]
We have 2016 rows per city.


### One Hot Encoding of categorical variables

In [12]:
numerical = [var for var in full_data[0].columns if full_data[0][var].dtype=='float64']
integer = [var for var in full_data[0].columns if full_data[0][var].dtype=='int64']
categorical = [var for var in full_data[0].columns if full_data[0][var].dtype=='O'] 

In [13]:
for loc_i in range(len(full_data)):
    full_data[loc_i] = pd.concat([full_data[loc_i][numerical+integer], pd.get_dummies(full_data[loc_i][categorical])], axis=1)

### Normalize Data

In [14]:
scaler = StandardScaler()

for loc_i in range(len(full_data)):
    full_data[loc_i][numerical] = scaler.fit_transform(full_data[loc_i][numerical])

In [ ]:
for i in range(len(full_data)):
    full_data[i] = full_data[i].astype(np.float32)

In [25]:
full_data[0].head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,cos_time,sin_time,RainToday,RainTomorrow,WindGustDir_E,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_E,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
2010-10-20,-0.193741,-0.003778,-0.302792,-0.472464,0.997965,-0.284700,-0.538669,-0.752841,0.237376,-0.267667,1.366063,1.293802,-1.188528,-0.804273,-0.073101,-0.006977,0.420689,-1.354426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-10-21,0.227810,-0.707366,-0.302792,-0.426408,-2.248505,-0.481092,-1.427704,-0.470078,1.804788,2.250345,1.279783,1.132882,1.519772,1.376807,-0.611082,-1.145110,0.443479,-1.345686,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-10-22,0.062200,-0.187323,-0.302792,-0.886964,0.493659,0.108084,0.054021,-0.470078,0.629229,0.669268,0.744852,0.614365,0.359072,0.504375,-0.126899,-0.247428,0.466117,-1.336555,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-10-23,-0.088353,0.164471,0.879402,-0.449436,0.934927,-1.364857,-0.242324,-0.752841,0.890465,-0.150550,0.296200,0.256767,-1.188528,0.504375,0.016563,0.217443,0.488593,-1.327036,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-10-24,0.212754,-1.120342,-0.302792,-0.495492,-2.311543,0.402672,-0.835014,-1.035603,1.282318,2.777371,-0.186964,0.381926,1.519772,1.376807,-0.126899,-1.369531,0.510903,-1.317130,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## B. LSTM_model Class

We will use LSTMs to predict the variable RainTomorrow using the data of a sequence of previous dates's features.

We create a LSTM class that will simplify the test of LSTM models on our dataset.

Later, we will be able to adjust the length of the sequence used to do a prediction, as well as the architecture of the model and the learning parameters.

In [55]:
class LSTM_model():
    def __init__(self, name='', full_data=full_data, prev_times=7):
        '''
        Input :     full_data.shape = (nb_cities, nb_days, nb_features)
                    prev_times : number of previous times to consider/ length of the sequence used by the LSTM model
        '''
        self.name = name
        self.prev_times = prev_times
        self.full_data = full_data
        self.data_transformed = False
        self.X_supervised = None
        self.y_supervised = None
        self.model = None
        self.cv = None
        self.history = []
        self.accuracy = []
        self.score = None
        self.model_checkpoint = ModelCheckpoint(f'best_{self.name}/', save_best_only=True, monitor='accuracy', mode='max')
        self.best_model = None

    # convert a dataframe to the "supervised learning" format
    def series_to_supervised(self, x, y, prev_times=7, dropnan=True):
        '''
        Input :     x.shape = (nb_days, nb_features)
                    y.shape = (nb_days,)
                    prev_times : number of previous times to consider

        Output :    x.shape = (nb_days-prev_times, (prev_times+1)*nb_features)
                    y.shape = (nb_days-prev_times,)
        '''
        n_vars = x.shape[1]
        cols, names = list(), list()
        # input sequence (t-prev_times, ..., t)
        for i in range(prev_times, 0, -1):
            cols.append(x.shift(i))
            names += [f'{x.columns[j]}(d-{i})' for j in range(n_vars)]
        cols.append(x)
        names += [f'{x.columns[j]}(d)' for j in range(n_vars)]
        # forecast RainTomorrow(t+1)
        cols.append(y)
        names += ['RainTommorow(d)']
        # put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)
        return agg.drop('RainTommorow(d)', axis=1), agg['RainTommorow(d)']
    
    #We transform the data to make it compatible with the LSTM model
    def transform_data(self):
        if self.data_transformed:
            return self.X_supervised, self.y_supervised
        X=[]
        y=[]
        for i in range(len(self.full_data)):
            X_city, y_city = self.full_data[i].drop('RainTomorrow', axis=1), self.full_data[i]['RainTomorrow']
            X_city, y_city = self.series_to_supervised(X_city, y_city, self.prev_times)
            X.append(X_city.values)
            y.append(y_city.values)
        X = np.array(X)
        y = np.array(y)
        X = X.reshape((X.shape[0]*X.shape[1], X.shape[2]))
        y = y.reshape((y.shape[0]*y.shape[1]))
        X = X.reshape((X.shape[0], self.prev_times+1, X.shape[1]//(self.prev_times+1)))
        self.X_supervised = X
        self.y_supervised = y
        self.data_transformed = True
    
    #Define a cross-validation strategy
    def define_cv(self, n_splits=5, shuffle=True, random_state=42):
        '''
        Input :     n_splits : number of folds
        Output :    cv : cross-validation strategy
        '''
        self.cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        return self.cv
    
    # design network
    def create_lstm_model(self,nb_layers=1,nb_neurons=[64], learning_rate=0.01, print_summary=True):
        '''
        Input :     input_shape : shape of the input
                    nb_layers : number of LSTM layers
                    nb_neurons : number of neurons in each LSTM layer
                    learning_rate : learning rate of the optimizer
        Output :    model : LSTM model
        '''
        if not self.data_transformed:
            self.transform_data()
        model = Sequential()
        for i in range(nb_layers):
            if i==0:
                model.add(LSTM(nb_neurons[i], activation='tanh', return_sequences=True, input_shape=(self.X_supervised.shape[1],self.X_supervised.shape[2])))
            elif i==nb_layers-1:
                model.add(LSTM(nb_neurons[nb_layers-1], activation='tanh', return_sequences=False))
            else:
                model.add(LSTM(nb_neurons[i], activation='tanh', return_sequences=True))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss=BinaryCrossentropy(), optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
        if print_summary:
            print(model.summary())
        return model
    
    #fit one LSTM model with (X_train, y_train)-(X_test, y_test) split
    def fit(self, X_train, X_test, y_train, y_test, epochs=10, batch_size=64, callbacks=None):
        '''
        Input :     callbacks : callbacks to use (for example to keep track of the best model during training)
        Output :    model : fitted LSTM model
        '''
        if self.model is None:
            raise ValueError('You must assign a model before fitting it')
        self.history.append(self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_data=(X_test, y_test)))
        
        return self.model
    
    # evaluate a model using cross-validation
    def evaluate(self, nb_layers=1, nb_neurons=[64], learning_rate= 0.01,n_splits=5,epochs=10, batch_size=64, random_state=42, shuffle=True, use_model_checkpoint=False, reset_model_checkpoint=True):
        '''
        Input :     (nb_layers, nb_neurons, learning_rate) : parameters of the LSTM model to evaluate
                    (epochs, batch_size) : parameters of the fit method
                    n_splits : number of folds for cross-validation
                    use_model_checkpoint : whether to register the best model during training or not
                    reset_model_checkpoint : whether to reset the model_checkpoint or not (if True, we return the score of the best model during training)
        Output :    score : accuracy score of the model (the one of the best model if reset_model_checkpoint=True & use_model_checkpoint=True, the one of the last model otherwise)
        '''
        self.transform_data() #We make sure that the data is transformed in the right format
        self.history = []
        self.accuracy = []
        callbacks=None
        print_summary = True
        #If we don't use cross-validation
        if n_splits==1:
            if reset_model_checkpoint and use_model_checkpoint: 
                self.initialize_model_checkpoint()
            if use_model_checkpoint:
                callbacks=[self.model_checkpoint]
            X_train, X_test, y_train, y_test = train_test_split(self.X_supervised, self.y_supervised, test_size=0.2, random_state=random_state)
            self.model = self.create_lstm_model(nb_layers=nb_layers, nb_neurons=nb_neurons, learning_rate=learning_rate)
            self.fit(X_train, X_test, y_train, y_test, epochs=epochs, batch_size=batch_size, callbacks=callbacks)
            if use_model_checkpoint:
                self.best_model = load_model(f'best_{self.name}/')
            if use_model_checkpoint and reset_model_checkpoint: #If we reset the model_checkpoint, we load the best model
                _, accuracy = self.best_model.evaluate(X_test, y_test, verbose=0)
            else:                                               #Otherwise, we load the last model
                _, accuracy = self.model.evaluate(X_test, y_test, verbose=0)
            self.score = accuracy
            print(f'Accuracy score: {100*self.score:.2f}')
        #If we use cross-validation
        elif n_splits>1:
            self.define_cv(n_splits=n_splits, random_state=random_state, shuffle=True)
            for train_index, test_index in self.cv.split(self.X_supervised, self.y_supervised):
                if reset_model_checkpoint and use_model_checkpoint: 
                    self.initialize_model_checkpoint()
                if use_model_checkpoint:
                    callbacks=[self.model_checkpoint]
                X_train, X_test = self.X_supervised[train_index], self.X_supervised[test_index]
                y_train, y_test = self.y_supervised[train_index], self.y_supervised[test_index] 
                self.model = self.create_lstm_model(nb_layers=nb_layers, nb_neurons=nb_neurons, learning_rate=learning_rate, print_summary=print_summary)
                self.fit(X_train, X_test, y_train, y_test, epochs=epochs, batch_size=batch_size, callbacks=callbacks)
                print_summary=False
                if use_model_checkpoint:
                    self.best_model = load_model(f'best_{self.name}/')
                if reset_model_checkpoint and use_model_checkpoint:
                    _, accuracy = self.best_model.evaluate(X_test, y_test, verbose=0)
                else:
                    _, accuracy = self.model.evaluate(X_test, y_test, verbose=0)
                self.accuracy.append(accuracy)
            self.score = np.mean(self.accuracy)
            print(f'Accuracy score: {100*self.score:.2f}')

    # plot history
    def plot_history(self, history, specific_label=''):
        plt.plot(history.history['accuracy'], label='accuracy_train')
        plt.plot(history.history['val_accuracy'], label='accuracy_test')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title(specific_label)
        plt.legend()

    # plot history of each training (for each fold)
    def plot_all_history(self, model='LSTM'):
        k = len(self.history)
        plt.figure(figsize=(15, 2*k))
        for i in range(k):
            plt.subplot(k//3+2, 3, i+1)
            self.plot_history(self.history[i], specific_label=f'fold {i+1}')
        plt.suptitle(model + f' Accuracy during training (prev_times={self.prev_times})', fontsize = '15', fontweight='bold')    
        plt.show()
    
    # Reset the model_checkpoint (to keep track of the best model during training)
    def initialize_model_checkpoint(self):
        self.model_checkpoint = ModelCheckpoint(f'best_{self.name}/', save_best_only=True, monitor='val_accuracy', mode='max')

## C. One Location

We keep the data of a single city.

### Data preparation

In [17]:
city = np.random.randint(0,22)
df = [full_data[city]]

### 14 DAYS SEQUENCE : Test on one Train-Test split (accuracy : 88.53%)

In [58]:
lstm_model_city = LSTM_model(name="lstm_model_city",full_data=df, prev_times=14)

In [59]:
lstm_model_city.evaluate(n_splits=1, learning_rate=0.001, epochs=6, batch_size=8, use_model_checkpoint=True)

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 64)                33792     
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 33857 (132.25 KB)
Trainable params: 33857 (132.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
197/201 [============================>.] - ETA: 0s - loss: 0.4401 - accuracy: 0.8204INFO:tensorflow:Assets written to: best_lstm_model_city\assets


INFO:tensorflow:Assets written to: best_lstm_model_city\assets


201/201 [==============================] - 12s 38ms/step - loss: 0.4383 - accuracy: 0.8207 - val_loss: 0.3552 - val_accuracy: 0.8479
Epoch 2/6
199/201 [============================>.] - ETA: 0s - loss: 0.3363 - accuracy: 0.8606INFO:tensorflow:Assets written to: best_lstm_model_city\assets


INFO:tensorflow:Assets written to: best_lstm_model_city\assets


201/201 [==============================] - 9s 47ms/step - loss: 0.3378 - accuracy: 0.8601 - val_loss: 0.3091 - val_accuracy: 0.8778
Epoch 3/6
201/201 [==============================] - 4s 19ms/step - loss: 0.3124 - accuracy: 0.8695 - val_loss: 0.3000 - val_accuracy: 0.8703
Epoch 4/6
199/201 [============================>.] - ETA: 0s - loss: 0.2936 - accuracy: 0.8763INFO:tensorflow:Assets written to: best_lstm_model_city\assets


INFO:tensorflow:Assets written to: best_lstm_model_city\assets


201/201 [==============================] - 7s 37ms/step - loss: 0.2929 - accuracy: 0.8770 - val_loss: 0.3029 - val_accuracy: 0.8853
Epoch 5/6
201/201 [==============================] - 3s 16ms/step - loss: 0.2788 - accuracy: 0.8819 - val_loss: 0.3088 - val_accuracy: 0.8778
Epoch 6/6
201/201 [==============================] - 3s 17ms/step - loss: 0.2635 - accuracy: 0.8863 - val_loss: 0.2995 - val_accuracy: 0.8778
Accuracy score: 88.53


In [70]:
print(lstm_model_city.name + f' accuracy score : {100*lstm_model_city.score:.2f}')

lstm_model_city accuracy score : 88.53


### 14 DAYS SEQUENCE : Cross-Validation (accuracy : 87.61%)

In [108]:
lstm_model_city_cv = LSTM_model(name="lstm_model_city_cv",full_data=df, prev_times=14)

In [109]:
lstm_model_city_cv.evaluate(n_splits=5, learning_rate=0.001, epochs=6, batch_size=8, use_model_checkpoint=True)

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_55 (LSTM)              (None, 64)                33792     
                                                                 
 dense_55 (Dense)            (None, 1)                 65        
                                                                 
Total params: 33857 (132.25 KB)
Trainable params: 33857 (132.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
196/201 [============================>.] - ETA: 0s - loss: 0.4519 - accuracy: 0.8170INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


201/201 [==============================] - 8s 25ms/step - loss: 0.4490 - accuracy: 0.8201 - val_loss: 0.3632 - val_accuracy: 0.8753
Epoch 2/6
201/201 [==============================] - 2s 10ms/step - loss: 0.3406 - accuracy: 0.8538 - val_loss: 0.2991 - val_accuracy: 0.8728
Epoch 3/6
196/201 [============================>.] - ETA: 0s - loss: 0.3097 - accuracy: 0.8680INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


201/201 [==============================] - 4s 22ms/step - loss: 0.3094 - accuracy: 0.8682 - val_loss: 0.3060 - val_accuracy: 0.8953
Epoch 4/6
201/201 [==============================] - 2s 10ms/step - loss: 0.2961 - accuracy: 0.8813 - val_loss: 0.3011 - val_accuracy: 0.8903
Epoch 5/6
201/201 [==============================] - 2s 10ms/step - loss: 0.2819 - accuracy: 0.8819 - val_loss: 0.3172 - val_accuracy: 0.8678
Epoch 6/6
201/201 [==============================] - 2s 11ms/step - loss: 0.2661 - accuracy: 0.8869 - val_loss: 0.3158 - val_accuracy: 0.8653
Epoch 1/6
200/201 [============================>.] - ETA: 0s - loss: 0.4123 - accuracy: 0.8188INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


...

201/201 [==============================] - 5s 24ms/step - loss: 0.3438 - accuracy: 0.8639 - val_loss: 0.3220 - val_accuracy: 0.8650
Epoch 3/6
201/201 [==============================] - 2s 11ms/step - loss: 0.3072 - accuracy: 0.8764 - val_loss: 0.3153 - val_accuracy: 0.8575
Epoch 4/6
201/201 [==============================] - 2s 11ms/step - loss: 0.2961 - accuracy: 0.8851 - val_loss: 0.2997 - val_accuracy: 0.8650
Epoch 5/6
199/201 [============================>.] - ETA: 0s - loss: 0.2867 - accuracy: 0.8863INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


201/201 [==============================] - 4s 22ms/step - loss: 0.2862 - accuracy: 0.8864 - val_loss: 0.2940 - val_accuracy: 0.8675
Epoch 6/6
195/201 [============================>.] - ETA: 0s - loss: 0.2690 - accuracy: 0.8917INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


INFO:tensorflow:Assets written to: best_lstm_model_city_cv\assets


201/201 [==============================] - 4s 22ms/step - loss: 0.2671 - accuracy: 0.8933 - val_loss: 0.2987 - val_accuracy: 0.8725
Accuracy score: 87.61


In [110]:
print(lstm_model_city_cv.name + f' accuracy score : {100*lstm_model_city_cv.score:.2f}')

lstm_model_city_cv accuracy score : 87.61


## D. Complete Dataset

### Data Preparation

A column is missing in full_data[12], so we add it !

In [17]:
#Identification de la colonne manquante
set(full_data[11].columns) - set(full_data[12].columns)	

{'WindGustDir_SW'}

In [18]:
#Ajout de la colonne manquante au bon emplacement
full_data[12].insert(32, 'WindGustDir_SW', 0)

### 14 DAYS SEQUENCE : Test on one Train-Test split (accuracy : 88.42%)

In [113]:
lstm_model = LSTM_model(name="lstm_model",full_data=full_data, prev_times=14)

In [114]:
lstm_model.evaluate(n_splits=1, learning_rate=0.001, epochs=6, batch_size=8, use_model_checkpoint=True)

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_60 (LSTM)              (None, 64)                33792     
                                                                 
 dense_60 (Dense)            (None, 1)                 65        
                                                                 
Total params: 33857 (132.25 KB)
Trainable params: 33857 (132.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
4405/4405 [==============================] - ETA: 0s - loss: 0.3073 - accuracy: 0.8737INFO:tensorflow:Assets written to: best_lstm_model\assets


INFO:tensorflow:Assets written to: best_lstm_model\assets


4405/4405 [==============================] - 51s 11ms/step - loss: 0.3073 - accuracy: 0.8737 - val_loss: 0.2881 - val_accuracy: 0.8766
Epoch 2/6
4399/4405 [============================>.] - ETA: 0s - loss: 0.2770 - accuracy: 0.8841INFO:tensorflow:Assets written to: best_lstm_model\assets


INFO:tensorflow:Assets written to: best_lstm_model\assets


4405/4405 [==============================] - 41s 9ms/step - loss: 0.2769 - accuracy: 0.8841 - val_loss: 0.2818 - val_accuracy: 0.8798
Epoch 3/6
4400/4405 [============================>.] - ETA: 0s - loss: 0.2630 - accuracy: 0.8888INFO:tensorflow:Assets written to: best_lstm_model\assets


INFO:tensorflow:Assets written to: best_lstm_model\assets


4405/4405 [==============================] - 41s 9ms/step - loss: 0.2630 - accuracy: 0.8887 - val_loss: 0.2740 - val_accuracy: 0.8815
Epoch 4/6
4405/4405 [==============================] - 44s 10ms/step - loss: 0.2473 - accuracy: 0.8951 - val_loss: 0.2791 - val_accuracy: 0.8788
Epoch 5/6
4404/4405 [============================>.] - ETA: 0s - loss: 0.2335 - accuracy: 0.9008INFO:tensorflow:Assets written to: best_lstm_model\assets


INFO:tensorflow:Assets written to: best_lstm_model\assets


4405/4405 [==============================] - 45s 10ms/step - loss: 0.2335 - accuracy: 0.9008 - val_loss: 0.2786 - val_accuracy: 0.8842
Epoch 6/6
4405/4405 [==============================] - 42s 10ms/step - loss: 0.2166 - accuracy: 0.9099 - val_loss: 0.3030 - val_accuracy: 0.8784
Accuracy score: 88.42


In [115]:
print(lstm_model.name + f' accuracy score : {100*lstm_model.score:.2f}')

lstm_model accuracy score : 88.42


### 14 DAYS SEQUENCE : Cross-Validation (accuracy : 88.72%)

In [30]:
lstm_model_cv = LSTM_model(name="lstm_model_cv",full_data=full_data, prev_times=14)

In [31]:
lstm_model_cv.evaluate(n_splits=5, learning_rate=0.001, epochs=6, batch_size=8, use_model_checkpoint=True)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 64)                33792     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33857 (132.25 KB)
Trainable params: 33857 (132.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
4402/4405 [============================>.] - ETA: 0s - loss: 0.3076 - accuracy: 0.8719INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 12s 3ms/step - loss: 0.3077 - accuracy: 0.8718 - val_loss: 0.2823 - val_accuracy: 0.8843
Epoch 2/6
4402/4405 [============================>.] - ETA: 0s - loss: 0.2802 - accuracy: 0.8828INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 11s 3ms/step - loss: 0.2802 - accuracy: 0.8828 - val_loss: 0.2715 - val_accuracy: 0.8861
Epoch 3/6
4389/4405 [============================>.] - ETA: 0s - loss: 0.2656 - accuracy: 0.8891INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


...

4405/4405 [==============================] - 12s 3ms/step - loss: 0.2776 - accuracy: 0.8841 - val_loss: 0.2857 - val_accuracy: 0.8791
Epoch 3/6
4394/4405 [============================>.] - ETA: 0s - loss: 0.2623 - accuracy: 0.8898INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 12s 3ms/step - loss: 0.2623 - accuracy: 0.8898 - val_loss: 0.2817 - val_accuracy: 0.8836
Epoch 4/6
4405/4405 [==============================] - 11s 3ms/step - loss: 0.2494 - accuracy: 0.8951 - val_loss: 0.2856 - val_accuracy: 0.8832
Epoch 5/6
4398/4405 [============================>.] - ETA: 0s - loss: 0.2346 - accuracy: 0.9007INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 12s 3ms/step - loss: 0.2347 - accuracy: 0.9007 - val_loss: 0.2769 - val_accuracy: 0.8845
Epoch 6/6
4405/4405 [==============================] - 11s 3ms/step - loss: 0.2202 - accuracy: 0.9071 - val_loss: 0.2781 - val_accuracy: 0.8812
Accuracy score: 88.72


In [32]:
print(lstm_model_cv.name + f' accuracy score : {100*lstm_model_cv.score:.2f}')

lstm_model_cv accuracy score : 88.72


### 14 DAYS SEQUENCE AND 2 LSTM LAYERS : Cross-Validation (88.61%)

In [43]:
lstm_model_cv = LSTM_model(name="lstm_model_cv",full_data=full_data, prev_times=14)

In [44]:
lstm_model_cv.evaluate(n_splits=5, learning_rate=0.001, nb_layers=2, nb_neurons=[64,32], epochs=6, batch_size=8, use_model_checkpoint=True)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_25 (LSTM)              (None, 15, 64)            33792     
                                                                 
 lstm_26 (LSTM)              (None, 32)                12416     
                                                                 
 dense_16 (Dense)            (None, 1)                 33        
                                                                 
Total params: 46241 (180.63 KB)
Trainable params: 46241 (180.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
4401/4405 [============================>.] - ETA: 0s - loss: 0.3096 - accuracy: 0.8705INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 23s 5ms/step - loss: 0.3096 - accuracy: 0.8705 - val_loss: 0.2826 - val_accuracy: 0.8832
Epoch 2/6
4401/4405 [============================>.] - ETA: 0s - loss: 0.2751 - accuracy: 0.8843INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 21s 5ms/step - loss: 0.2750 - accuracy: 0.8844 - val_loss: 0.2756 - val_accuracy: 0.8834
Epoch 3/6
4399/4405 [============================>.] - ETA: 0s - loss: 0.2563 - accuracy: 0.8932INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


...

4405/4405 [==============================] - 22s 5ms/step - loss: 0.2522 - accuracy: 0.8933 - val_loss: 0.2756 - val_accuracy: 0.8810
Epoch 4/6
4401/4405 [============================>.] - ETA: 0s - loss: 0.2301 - accuracy: 0.9036INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 21s 5ms/step - loss: 0.2303 - accuracy: 0.9035 - val_loss: 0.2822 - val_accuracy: 0.8811
Epoch 5/6
4405/4405 [==============================] - 19s 4ms/step - loss: 0.2084 - accuracy: 0.9134 - val_loss: 0.2883 - val_accuracy: 0.8780
Epoch 6/6
4399/4405 [============================>.] - ETA: 0s - loss: 0.1826 - accuracy: 0.9255INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


INFO:tensorflow:Assets written to: best_lstm_model_cv/assets


4405/4405 [==============================] - 21s 5ms/step - loss: 0.1826 - accuracy: 0.9255 - val_loss: 0.2978 - val_accuracy: 0.8815
Accuracy score: 88.61


In [45]:

print(lstm_model_cv.name + f' accuracy score : {100*lstm_model_cv.score:.2f}')

lstm_model_cv accuracy score : 88.61


### 30 DAYS SEQUENCE AND 3 LSTM LAYERS : Test on one Train-Test split (accuracy : 88.99%)

In [56]:
lstm_model_30 = LSTM_model(name="lstm_model_30",full_data=full_data, prev_times=30)

In [57]:
lstm_model_30.evaluate(n_splits=1, learning_rate=0.001, nb_layers=3, nb_neurons=[64,64,32], epochs=10, batch_size=8, use_model_checkpoint=True)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_44 (LSTM)              (None, 31, 64)            33792     
                                                                 
 lstm_45 (LSTM)              (None, 31, 64)            33024     
                                                                 
 lstm_46 (LSTM)              (None, 32)                12416     
                                                                 
 dense_25 (Dense)            (None, 1)                 33        
                                                                 
Total params: 79265 (309.63 KB)
Trainable params: 79265 (309.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
4368/4370 [============================>.] - ETA: 0s - loss: 0.3148 - accuracy: 0.8699INFO:tensorflow:Assets written to: best_lstm_

INFO:tensorflow:Assets written to: best_lstm_model_30/assets


4370/4370 [==============================] - 59s 13ms/step - loss: 0.3148 - accuracy: 0.8699 - val_loss: 0.2794 - val_accuracy: 0.8833
Epoch 2/10
4367/4370 [============================>.] - ETA: 0s - loss: 0.2714 - accuracy: 0.8865INFO:tensorflow:Assets written to: best_lstm_model_30/assets


INFO:tensorflow:Assets written to: best_lstm_model_30/assets


4370/4370 [==============================] - 59s 14ms/step - loss: 0.2715 - accuracy: 0.8864 - val_loss: 0.2799 - val_accuracy: 0.8859
Epoch 3/10
4370/4370 [==============================] - ETA: 0s - loss: 0.2496 - accuracy: 0.8943INFO:tensorflow:Assets written to: best_lstm_model_30/assets


INFO:tensorflow:Assets written to: best_lstm_model_30/assets


4370/4370 [==============================] - 60s 14ms/step - loss: 0.2496 - accuracy: 0.8943 - val_loss: 0.2651 - val_accuracy: 0.8899
Epoch 4/10
4370/4370 [==============================] - 56s 13ms/step - loss: 0.2287 - accuracy: 0.9047 - val_loss: 0.2696 - val_accuracy: 0.8850
Epoch 5/10
4370/4370 [==============================] - 56s 13ms/step - loss: 0.2059 - accuracy: 0.9150 - val_loss: 0.2775 - val_accuracy: 0.8874
Epoch 6/10
4370/4370 [==============================] - 55s 13ms/step - loss: 0.1774 - accuracy: 0.9273 - val_loss: 0.2900 - val_accuracy: 0.8829
Epoch 7/10
4370/4370 [==============================] - 55s 12ms/step - loss: 0.1527 - accuracy: 0.9395 - val_loss: 0.3293 - val_accuracy: 0.8871
Epoch 8/10
4370/4370 [==============================] - 55s 12ms/step - loss: 0.1273 - accuracy: 0.9515 - val_loss: 0.3588 - val_accuracy: 0.8730
Epoch 9/10
4370/4370 [==============================] - 56s 13ms/step - loss: 0.1054 - accuracy: 0.9605 - val_loss: 0.3711 - val_accura

In [58]:
print(lstm_model_30.name + f' accuracy score : {100*lstm_model_30.score:.2f}')

lstm_model_30 accuracy score : 88.99


## Own exploration

In [ ]:
def test_my_lstm_model(my_prev_times, my_nb_layers, my_nb_neurons, my_epochs, my_batch_size, my_learning_rate, my_n_splits):
    my_lstm_model = LSTM_model(name="my_lstm_model",full_data=full_data, prev_times=my_prev_times)
    my_lstm_model.evaluate(n_splits=my_n_splits, learning_rate=my_learning_rate, nb_layers=my_nb_layers, nb_neurons=my_nb_neurons, epochs=my_epochs, batch_size=my_batch_size, use_model_checkpoint=True)
    print(my_lstm_model.name + f' accuracy score : {100*my_lstm_model.score:.2f}')

Use the below commented code to try your own LSTM model

First, define your parameters :

In [ ]:
my_prev_times = ...     #To complete with the sequence length you want to use

'''Network parameters'''

my_nb_layers = ...      #To complete with the number of LSTM layers you want to use
my_nb_neurons = ...     #To complete with the number of neurons you want to use, it is a list of length my_nb_layers

'''Training parameters'''

my_epochs = ...         #To complete with the number of epochs you want to use
my_batch_size = ...     #To complete with the batch size you want to use
my_learning_rate = ...  #To complete with the learning rate you want to use

'''Validation parameters'''

my_n_splits = ...       #To complete with the number of folds you want to use

Then, run the below cell to train your model and have your accuracy score.

In [ ]:
test_my_lstm_model(my_prev_times, my_nb_layers, my_nb_neurons, my_epochs, my_batch_size, my_learning_rate, my_n_splits)